In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import argparse
from src.gelos_config import GELOSConfig
from src.downloader import Downloader
from src.aoi_processor import AOI_Processor
from pathlib import Path
import shutil

In [4]:
config = "/home/benchuser/code/config.yml"
gelosconfig = GELOSConfig.from_yaml(config)

working_directory = Path(gelosconfig.directory.working) / gelosconfig.dataset.version
# create working directory with version number if none exists
working_directory.mkdir(exist_ok=True)
# copy yaml to working directory
shutil.copy(config, working_directory / "config.yaml")


PosixPath('/home/benchuser/data/v0.40.2test/config.yaml')

In [5]:
vars(gelosconfig)


{'dataset': DatasetConfig(version='v0.40.2test'),
 'aoi': AoiConfig(version='v0.30'),
 'directory': DirectoryConfig(working='/home/benchuser/data', output='/home/benchuser/final_data'),
 'log_errors': True,
 'sentinel_2': Sentinel2Config(collection='sentinel-2-l2a', resolution=10, native_crs=True, fill_na=False, na_value=-999, dtype=dtype('int16'), time_ranges=['2023-01-01/2023-03-31', '2023-04-01/2023-06-30', '2023-07-01/2023-09-30', '2023-10-01/2023-12-31'], nodata_pixel_percentage=5, cloud_cover=50, cloud_band='SCL', bands=['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'SCL']),
 'sentinel_1': Sentinel1Config(collection='sentinel-1-rtc', resolution=10, native_crs=False, fill_na=False, na_value=-999, dtype=dtype('float32'), nodata_pixel_percentage=5, delta_days=12, bands=['vv', 'vh']),
 'landsat': LandsatConfig(collection='landsat-c2-l2', resolution=30, native_crs=False, fill_na=False, na_value=-999, dtype=dtype('float32'), platforms=['landsat-8',

In [7]:
downloader = Downloader(gelosconfig)

/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34815 instead
  warnings.warn(


In [ ]:
vars(downloader)

{'config': GELOSConfig(dataset=DatasetConfig(version='v0.40.2test'), aoi=AoiConfig(version='v0.30'), directory=DirectoryConfig(working='/home/benchuser/data', output='/home/benchuser/final_data'), log_errors=True, sentinel_2=Sentinel2Config(collection='sentinel-2-l2a', resolution=10, native_crs=True, fill_na=False, na_value=-999, dtype=dtype('int16'), time_ranges=['2023-01-01/2023-03-31', '2023-04-01/2023-06-30', '2023-07-01/2023-09-30', '2023-10-01/2023-12-31'], nodata_pixel_percentage=5, cloud_cover=50, cloud_band='SCL', bands=['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'SCL']), sentinel_1=Sentinel1Config(collection='sentinel-1-rtc', resolution=10, native_crs=False, fill_na=False, na_value=-999, dtype=dtype('float32'), nodata_pixel_percentage=5, delta_days=12, bands=['vv', 'vh']), landsat=LandsatConfig(collection='landsat-c2-l2', resolution=30, native_crs=False, fill_na=False, na_value=-999, dtype=dtype('float32'), platforms=['landsat-8', 'lan

In [27]:
processor = AOI_Processor(
    aoi_index = 122,
    aoi = downloader.aoi_gdf.iloc[122],
    chip_index = downloader.chip_index,
    working_directory = downloader.working_directory,
    catalog = downloader.catalog,
    config = gelosconfig,
)

In [28]:
%pdb on
new_chip_gdf = processor.process_aoi()

Automatic pdb calling has been turned ON

Processing AOI at index 122
Searching Sentinel-2 scenes for 2023-01-01/2023-03-31
Searching Sentinel-2 scenes for 2023-04-01/2023-06-30
Searching Sentinel-2 scenes for 2023-07-01/2023-09-30
Searching Sentinel-2 scenes for 2023-10-01/2023-12-31
searching sentinel_1 and landsat scenes close to 2023-01-12 09:52:49.024000+00:00 within 2023-01-01/2023-03-31


/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/pystac_client/item_search.py:850: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


searching sentinel_1 and landsat scenes close to 2023-04-22 09:50:29.024000+00:00 within 2023-04-01/2023-06-30
searching sentinel_1 and landsat scenes close to 2023-09-16 09:36:19.024000+00:00 within 2023-07-01/2023-09-30
searching sentinel_1 and landsat scenes close to 2023-12-05 09:42:59.024000+00:00 within 2023-10-01/2023-12-31
searching land cover data...
searching dem data...
stacking landsat data...


/opt/conda/envs/gfm_bench/lib/python3.12/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


AttributeError: 'DataArray' object has no attribute 'rio'

> /opt/conda/envs/gfm_bench/lib/python3.12/site-packages/xarray/core/common.py(306)__getattr__()
    304                 with suppress(KeyError):
    305                     return source[name]
--> 306         raise AttributeError(
    307             f"{type(self).__name__!r} object has no attribute {name!r}"
    308         )

